In [1]:
import os, sys
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# path 설정
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(""))))
print(sys.path[-1])

d:\base\boostcamp\apart\level2-competitiveds-recsys-06


In [2]:
raw_path: str = "../../data/raw"
processed_path: str = "../../data/processed"
submission_path: str = "../../data/submission"
pd.set_option('display.max_columns', None)

train: pd.DataFrame = pd.read_csv(os.path.join(raw_path,"train.csv"))
test: pd.DataFrame = pd.read_csv(os.path.join(raw_path,"test.csv"))
interest: pd.DataFrame = pd.read_csv(os.path.join(raw_path,"interestRate.csv"))
park: pd.DataFrame = pd.read_csv(os.path.join(raw_path,"parkInfo.csv"))
school: pd.DataFrame = pd.read_csv(os.path.join(raw_path,"schoolinfo.csv"))
subway: pd.DataFrame = pd.read_csv(os.path.join(raw_path,"subwayInfo.csv"))

train["_type"] = "train"
test["_type"] = "test"
train['original_index'] = train.index
test['original_index'] = test.index

In [3]:
df = pd.concat([train, test], axis=0)

In [4]:
v4 = pd.read_csv(os.path.join(processed_path, "v4_baek+recent+rank.csv"))
# ARIMA 인덱스 추가
from src.pre_process.deposit.arima_deposit_index import ArimaDepositIndexFeature
interest = pd.read_csv(os.path.join(raw_path, "interestRate.csv"))
v4_arima = ArimaDepositIndexFeature(v4, interest).get_data()
v4_arima

d:\Program\Dev\ANACONDA\envs\apart\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['typ']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)


,index,area_m2,contract_year_month,contract_day,contract_type,floor,built_year,latitude,longitude,age,deposit,_type,apt_idx,area,area_price,area_m2_price,grid_id,nearest_subway_distance,nearest_subway_idx,num_subway_within_1km,category_interchange_within_1km,num_subway_within_500m,category_interchange_within_500m,nearest_elementary_school_distance,nearest_elementary_school_within_1km,has_elementary_school_within_1km,nearest_middle_school_distance,nearest_middle_school_within_1km,has_middle_school_within_1km,nearest_high_school_distance,nearest_high_school_within_1km,has_high_school_within_1km,nearest_park_distance,nearest_park_within_500.0m,has_park_within_500.0m,apt_deposit_rank,apt_area_deposit_rank,original_index,recent_deposit,interest_rate,diff_interest_rate,arima_deposit_index
0,0,84.9981,201906,25,2,9,2019,37.054314,127.045216,0,17000.0,train,0,25.8,658.914729,200.004471,25581.836691,717,40,1,1,0,0,156.120431,1,1,465.124904,2,1,990.854810,1,1,498.618918,1,1,7683,29300,0,NaN,1.78,-0.07,102.180354
1,1,84.9981,202003,26,2,20,2019,37.054314,127.045216,1,23000.0,train,0,25.8,891.472868,270.594284,25581.836691,717,40,1,1,0,0,156.120431,1,1,465.124904,2,1,990.854810,1,1,498.618918,1,1,7683,29300,1,17000.0,1.26,-0.17,105.586089
2,2,84.9981,202003,28,2,8,2019,37.054314,127.045216,1,23000.0,train,0,25.8,891.472868,270.594284,25581.836691,717,40,1,1,0,0,156.120431,1,1,465.124904,2,1,990.854810,1,1,498.618918,1,1,7683,29300,2,23000.0,1.26,-0.17,105.586089
3,3,59.3400,201907,15,2,1,1986,36.964647,127.055847,33,5000.0,train,1,18.0,277.777778,84.260195,18375.213675,3897,41,0,0,0,0,214.559689,2,1,688.046835,1,1,644.366430,1,1,169.839678,3,1,18366,58901,3,NaN,1.68,-0.10,103.855832
4,4,59.8100,201904,12,2,6,1995,36.972390,127.084514,24,1800.0,train,2,18.1,99.447514,30.095302,12627.879781,2039,41,0,0,0,0,1708.489263,0,0,2197.946485,0,0,2264.821766,0,0,382.401815,1,1,18369,58863,4,NaN,1.85,-0.09,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1951395,150167,115.5101,202402,27,0,17,2010,37.528394,126.659398,14,NaN,test,17869,35.0,NaN,NaN,33310.656662,1483,650,0,0,0,0,313.128782,2,1,482.435625,3,1,224.754177,2,1,398.113485,1,1,6088,25360,150167,37000.0,3.62,-0.04,142.092659
1951396,150168,142.8738,202403,2,0,4,2010,37.528394,126.659398,14,NaN,test,17869,43.3,NaN,NaN,33310.656662,1483,650,0,0,0,0,313.128782,2,1,482.435625,3,1,224.754177,2,1,398.113485,1,1,6088,17911,150168,43000.0,3.59,-0.03,142.295862
1951397,150169,142.8738,202403,16,1,13,2010,37.528394,126.659398,14,NaN,test,17869,43.3,NaN,NaN,33310.656662,1483,650,0,0,0,0,313.128782,2,1,482.435625,3,1,224.754177,2,1,398.113485,1,1,6088,17911,150169,43000.0,3.59,-0.03,142.295862
1951398,150170,114.9285,202403,22,1,2,2010,37.528394,126.659398,14,NaN,test,17869,34.8,NaN,NaN,33310.656662,1483,650,0,0,0,0,313.128782,2,1,482.435625,3,1,224.754177,2,1,398.113485,1,1,6088,24128,150170,32500.0,3.59,-0.03,142.295862


In [8]:
# _type 컬럼을 제외한 데이터프레임 생성
v4_arima_excl_type = v4_arima.drop(columns=['_type'])

# deposit과의 상관관계 계산
correlation_with_deposit = v4_arima_excl_type.corr()['area_price']

# 상관관계 출력
correlation_with_deposit.sort_values(ascending=False)

area_price                              1.000000
area_m2_price                           0.999992
deposit                                 0.784147
grid_id                                 0.722829
recent_deposit                          0.712262
num_subway_within_1km                   0.419536
category_interchange_within_1km         0.397362
num_subway_within_500m                  0.311769
category_interchange_within_500m        0.297265
index                                   0.283101
original_index                          0.283101
apt_idx                                 0.280750
built_year                              0.183334
longitude                               0.179188
nearest_high_school_within_1km          0.166885
nearest_middle_school_within_1km        0.157445
arima_deposit_index                     0.153830
has_middle_school_within_1km            0.132537
nearest_elementary_school_within_1km    0.115233
latitude                                0.109493
floor               

In [9]:
v4_arima['infra_score'] = (
    v4_arima['num_subway_within_1km'] * 0.419536 +
    v4_arima['nearest_high_school_within_1km'] * 0.166885 +
    v4_arima['nearest_middle_school_within_1km'] * 0.157445 +
    v4_arima['nearest_elementary_school_within_1km'] * 0.115233 +
    v4_arima['nearest_park_within_500.0m'] * 0.031857
)

v4_arima['infra_score'].describe()

count    1.951400e+06
mean     1.751156e+00
std      9.224448e-01
min      0.000000e+00
25%      1.038278e+00
50%      1.688048e+00
75%      2.339330e+00
max      5.774712e+00
Name: infra_score, dtype: float64

In [10]:
v4_arima.to_csv(os.path.join(processed_path, "v7_infra_score.csv"), index=False)

In [11]:
# _type 컬럼을 제외한 데이터프레임 생성
v4_arima_excl_type = v4_arima.drop(columns=['_type'])

# deposit과의 상관관계 계산
correlation_with_deposit = v4_arima_excl_type.corr()['deposit']

# 절대값이 높은 순으로 정렬
correlation_with_deposit_sorted = correlation_with_deposit.abs().sort_values(ascending=False)

# 상관관계 출력
correlation_with_deposit_sorted

deposit                                 1.000000
recent_deposit                          0.927089
apt_area_deposit_rank                   0.800137
area_price                              0.784147
area_m2_price                           0.784008
apt_deposit_rank                        0.744761
grid_id                                 0.676606
area                                    0.521651
area_m2                                 0.521623
num_subway_within_1km                   0.314987
category_interchange_within_1km         0.292699
infra_score                             0.242154
num_subway_within_500m                  0.221344
apt_idx                                 0.220488
original_index                          0.217950
index                                   0.217950
nearest_subway_distance                 0.216313
category_interchange_within_500m        0.208316
longitude                               0.157374
built_year                              0.144960
age                 